In [1]:
from smach_utils2 import *
import matplotlib.pyplot as plt
#arm = moveit_commander.MoveGroupCommander('arm')


/home/roboworks/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
arm = moveit_commander.MoveGroupCommander('arm')

[ WARN] [1710957224.758793811]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1710957224.761449623]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1710957224.761486135]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1710957224.762542975]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1710957224.762580313]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1710957225.220999743]: Kinematics solver doesn't support #attempts any

In [ ]:
head.set_joint_values([0.0,-0.65])
arm.set_named_target('go')
arm.go()

In [ ]:

rospy.sleep(0.6)
rospy.sleep(0.6)
image= cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)

In [ ]:

plt.imshow(debug_image)

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    tf_man.pub_static_tf(pos=[res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z], rot=[0,0,0,1],ref="head_rgbd_sensor_rgb_frame",point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])



In [ ]:
omni_base.move_d_to(target_distance=0.78,target_link='apple')

In [ ]:
target_object='apple'
trans,_=tf_man.getTF(target_object,ref_frame="base_link")
trans


In [ ]:
target_object='apple'
_,rot= tf_man.getTF("base_link",ref_frame='map')

original_rot=tf.transformations.euler_from_quaternion(rot)[2]
trans,_=tf_man.getTF(target_object,ref_frame="base_link")


trans , original_rot

In [ ]:
_,rot= tf_man.getTF("base_link",ref_frame='map')
original_rot=tf.transformations.euler_from_quaternion(rot)[2]
target_object='apple'


succ = False
            
while not succ:
    
    _,rot= tf_man.getTF("base_link",ref_frame='map')
    trans,_=tf_man.getTF(target_object,ref_frame="base_link")

    trans
    eX, eY, eZ = trans
    
    eX+= -0.4
    eY+= -.06
    
    eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
    print (eT)
    if eT > np.pi: eT=-2*np.pi+eT
    if eT < -np.pi: eT= 2*np.pi+eT
    rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
    X, Y, Z = trans
    rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
    
    if abs(eX) <=0.05 :
        print ('here')
        eX = 0
    if abs(eY) <=0.05  :
        eY = 0
    if abs(eT   ) < 0.1:
        eT = 0
    succ =  eX == 0 and eY == 0 and eT==0

    
    omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 


In [ ]:
res.names

In [ ]:
hand_cam= HAND_RGB()


In [ ]:
#clear_octo_client()

pickup_pose=[0.65,-1.2,0.0,-1.9, 0.0, 0.0]
succ= arm.go(pickup_pose)
gripper.open()


In [ ]:

image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
for i in range(len(res.names)):
    if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
        bbM=res.pt_min.data[2*i:2*i+2]
        bbm=res.pt_max.data[2*i:2*i+2]
        bbx=[bbM,bbm]
print(bbx)
plt.imshow(debug_image)

# [(297, 191), (231, 226)]  at pkcp pose #Real Tak
### [(323, 266), (280, 215)]  at pkcp pose #Gazebo


In [ ]:
eX,eY= 0.3,0.3 

while abs(eX) > 0.05 or abs(eY) > 0.05:
    image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
    img_msg  = bridge.cv2_to_imgmsg(image)
    req      = classify_client.request_class()
    req.in_.image_msgs.append(img_msg)
    res      = classify_client(req)


    for i in range(len(res.names)):
        if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
            bbM=res.pt_min.data[2*i:2*i+2]
            bbm=res.pt_max.data[2*i:2*i+2]
            bbx=[bbM,bbm]
    print(bbx)
    eY=(bbx[1][1]-215)/215    #experience value
    #print(f'error relative to gripper cam {eY}')


    #eX=(bbx[0][0]-323)/323
    eY=(bbx[1][1]-239)/239
    eX=(bbx[0][0]-312)/312
    #eY=(bbx[1][1]-215)/215    #experience value

    print(f'error relative to gripper cam ex={eX} ey={eY}')
    omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 
print(f'error relative to gripper cam ex={eX} ey={eY}:::::: Error within tolerance')

In [ ]:
bbx

In [ ]:
eY=(bbx[1][1]-239)/239
eX=(bbx[0][0]-312)/312
print(f'error relative to gripper cam ex={eX} ey={eY}')


In [ ]:
#omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 


In [ ]:
clear_octo_client()

av=arm.get_current_joint_values()
print (av,'av')
av[0]+= -0.17###LAB
#av[0]+= -0.35###GAZ
succ=arm.go(av)
rospy.sleep(0.5)
gripper.close(force=0.06)

In [ ]:
av=arm.get_current_joint_values()
print (av,'av')
av[0]+= 0.15
succ=arm.go(av)
omni_base.tiny_move(velX=-0.3, std_time=4.0)
arm.set_named_target('go')
arm.go()
#tf_man.pub_static_tf(point_name='apple')

In [ ]:
plt.imshow(hand_cam.get_image())

In [ ]:
image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
plt.imshow(debug_image)

In [ ]:
gripper.open()

In [ ]:
gripper.close(force=0.06)

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
    tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])




In [ ]:
def detect_object_yolo(object_name,res):
    # find object_name in the response message from object_classification service (Yolo)
    objs=[]
    for i,name in enumerate(res.names):
        objs.append(name.data[4:])
        if name.data[4:]==object_name:return res.poses[i]
    if object_name=='all':
        
        return objs
    return []

In [ ]:
#img_msg  = bridge.cv2_to_imgmsg(rgbd.get_image())  #REAL
img_msg  = bridge.cv2_to_imgmsg(cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR ))#GAZEB
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
objects=detect_object_yolo('all',res)   
if len (objects)!=0:
    for i in range(len(res.poses)):
        tf_man.getTF("head_rgbd_sensor_rgb_frame")
        position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
        tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
        rospy.sleep(0.3)
        tf_man.change_ref_frame_tf(res.names[i].data[4:])
        rospy.sleep(0.3)
        pose , _=tf_man.getTF(res.names[i].data[4:])
        new_row = {'x': pose[0], 'y': pose[1], 'z': pose[2], 'obj_name': res.names[i].data[4:]}
        objs.loc[len(objs)] = new_row
objs

In [ ]:
objs[['x','y','z']].values

In [ ]:
#get closest object
rob_pos,_=tf_man.getTF('base_link')
np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))
objs.iloc[np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))]['obj_name']

In [ ]:
objs[objs['obj_name']==objs.obj_name.values[0]].x.values[0]

# PLANE 1
#### 7.883384	5.948245	0.701945,
#### 8.214565	5.962619	0.700034,
#### 8.213697	6.107727	0.709487,
####  7.893623	6.082935	0.693285

# PLANE 2
#### 7.88076	6.086619	0.416096,
#### 8.214128	5.9663	0.416672

# Plane 3
#### 8.20238	5.963262	0.117541

In [ ]:
head.set_named_target('neutral')

av=arm.get_current_joint_values()
av[0]=0.5
av[1]=-0.5
arm.go(av)
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:


av=arm.get_current_joint_values()
av[0]=0.05

arm.go(av)

In [ ]:
head.set_joint_values([-np.pi/2 , -0.8])

av=arm.get_current_joint_values()
av[0]=0.0
av[1]=-0.5
arm.go(av)

In [4]:
import pandas as pd


In [19]:
import pandas as pd
objs = pd.read_csv ('/home/roboworks/Documents/objs.csv')
objs=objs.drop(columns='Unnamed: 0')

In [20]:
objs

,x,y,z,obj_name,shelves,pickup,category
0,8.540362,-2.302229,0.711838,baseball,False,False,balls
1,8.588709,-2.163747,0.710513,f_cups,False,False,kitchen
2,8.159609,-1.997206,0.006306,spatula,False,True,kitchen
3,6.250804,-0.725341,0.831389,a_marbles,False,False,other
4,7.309552,-3.363026,0.677819,a_marbles,False,False,other
5,8.160490,-2.318883,0.673028,pitcher_lid,False,False,other
6,6.095096,-3.146115,0.258413,master_chef_can,False,False,food
7,8.410027,-2.522980,0.155381,master_chef_can,False,False,food
8,6.402390,-0.752308,0.761527,a_pitcher_base,False,False,other
9,7.283742,-3.308466,0.173525,windex_bottle,False,False,other


In [44]:
from sklearn.cluster import KMeans
shelf_objs=objs[objs['shelves']==True ]
sh_o_btm=shelf_objs[shelf_objs['z']<0.2]


In [82]:
'other' in sh_o_top['category'].values

True

In [91]:
shelf_objs=objs[objs['shelves']==True ]

sh_o_btm=shelf_objs[shelf_objs['z']<0.2]
sh_o_mdl=shelf_objs[(shelf_objs['z']>0.2 )& (shelf_objs['z']<0.5) ]
sh_o_top=shelf_objs[(shelf_objs['z']>0.5 )& (shelf_objs['z']<1.5) ]
sh_o_btm['category'].value_counts(),sh_o_mdl['category'].value_counts() 

a= sh_o_btm['category'].value_counts()
if 'other' in sh_o_btm['category'].values:a.drop('other', inplace=True)
print(f'btm shelf category {a.index[a.argmax()]}')

a= sh_o_mdl['category'].value_counts()
if 'other' in sh_o_mdl['category'].values:a.drop('other', inplace=True)
print(f'middle shelf category {a.index[a.argmax()]}')


a= sh_o_top['category'].value_counts()
if 'other' in sh_o_top['category'].values:a.drop('other', inplace=True)
print(f'top shelf category {a.index[a.argmax()]}')


btm shelf category tools
middle shelf category kitchen
top shelf category kitchen


In [86]:
a

category
tools    1
Name: count, dtype: int64

In [3]:
kitchen =['bowl','spatula','spoon', 'bowl']
tools=['extra_large_clamp','large_clamp','small_clamp','medium_clamp','adjustable_wrench',
'flat_screwdriver','phillips_screwdriver','wood_block']
balls= ['softball','tennis_ball']
fruits= ['apple','banana', 'lemon','pear']
food =['chips_can'
,'mustard_bottle'
,'potted_meat_can'
,'tomato_soup_can'
,'tuna_fish_can']

def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats   


In [4]:
def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
    
    

In [5]:
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats

In [6]:
objs

,x,y,z,obj_name,category
0,5.980992,4.677664,0.506293,a_colored_wood_blocks,other
1,6.219399,4.659809,0.421303,bowl,kitchen
2,5.928290,4.551338,0.290167,b_colored_wood_blocks,other
3,6.390869,4.590297,0.431736,a_pitcher_base,other
4,8.230576,6.101967,0.704379,master_chef_can,other
5,8.221585,5.984989,0.944968,nine_hole_peg_test,other
6,8.032595,5.981796,0.418046,h_cups,other
7,7.947621,6.034316,0.421347,softball,balls
8,8.154552,5.987593,0.688931,tuna_fish_can,food
9,8.119175,6.098844,0.087389,large_clamp,tools


In [7]:
import numpy as np
regions={'shelves':np.load('/home/roboworks/Documents/shelf_sim.npy'),'pickup':np.load('/home/roboworks/Documents/pickup_sim.npy')}
def is_inside(x,y):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x)) 
for name in regions:
    in_region=[]
    area_box=regions[name]
    print (area_box,name)
    for index, row in objs[['x','y','z']].iterrows():
        in_region.append(is_inside(row.x, row.y))
        print (row.x, row.y,is_inside(row.x, row.y))
    objs[name]=pd.Series(in_region)
objs

[[8.  6.2]
 [8.6 5.9]] shelves
8.427302439444869 -2.387099938691619 False
4.189883221764755 -3.12817517224511 False
6.996095572401662 -3.385444536385085 False
8.241146517384646 -2.1234714397951437 False
8.385500617275074 -2.6580331149631853 False
6.738603323946117 -3.3399815724440334 False
8.456481766905478 -2.1191276737283378 False
8.412992079952813 -2.376650791173282 False
7.408113251719342 -3.5837193368711446 False
9.827777536725833 -0.1736827784155536 False
8.71789504057108 -0.7650194610032485 False
9.42270083077709 -0.7967892993931813 False
8.72939391900003 -0.8863480440762128 False
9.679823610875626 -0.6422716938996136 False
9.6812992225024 -0.6301467986524352 False
9.599156645696697 -0.5458299868701944 False
9.66885550110473 -0.8080967243181236 False
8.94006119969579 -0.8552727450276483 False
9.427913316426327 -0.8005868059264166 False
8.941879617700575 -0.8571746001411781 False
9.464883890218273 -0.6275242023490577 False
8.941935417049475 -0.8579549063104037 False
9.75603472185

,x,y,z,obj_name,shelves,pickup,category
0,8.427302,-2.387100,0.706535,potted_meat_can,False,False,food
1,4.189883,-3.128175,0.344880,a_toy_airplane,False,False,other
2,6.996096,-3.385445,0.032005,pitcher_lid,False,False,other
3,8.241147,-2.123471,0.722802,tomato_soup_can,False,False,food
4,8.385501,-2.658033,0.125929,master_chef_can,False,False,food
5,6.738603,-3.339982,0.025703,pitcher_lid,False,False,other
6,8.456482,-2.119128,0.693669,racquetball,False,False,balls
7,8.412992,-2.376651,0.697793,potted_meat_can,False,False,food
8,7.408113,-3.583719,0.248668,a_colored_wood_blocks,False,False,other
9,9.827778,-0.173683,1.062830,pudding_box,False,False,food


In [42]:
(area_box[:,1].max() > row.y),(area_box[:,1].min() < row.y),((area_box[:,0].max() > row.x) , (area_box[0,0].min() < row.x)) 

(True, True, (True, True))

In [23]:
def is_inside(x,y):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x)) 

,x,y,z


In [9]:
pickup_region=np.zeros((2,2))
#np.save('/home/roboworks/Documents/pickup_region.npy', pickup_region)

In [30]:
ab[0,0],ab[0,1]=6,6
ab[1,0],ab[1,1]=7,4.0

In [1]:
import numpy as np

#ab=np.load('/home/roboworks/Documents/pickup_region.npy')
ab= np.load('/home/roboworks/Documents/pickup_sim.npy')

In [4]:

ab[0,0]=5.8
ab

array([[5.8, 6. ],
       [7. , 4. ]])

In [35]:
np.save('/home/roboworks/Documents/pickup_region.npy',ab)


In [5]:
#np.save('/home/roboworks/Documents/shelf_sim.npy',ab)
np.save('/home/roboworks/Documents/pickup_sim.npy',ab)

In [93]:
objs[objs['pickup']==True][['x','y','z']].values

array([[6.17398135, 4.65753106, 0.42089542],
       [6.36147013, 4.59256394, 0.44251327],
       [6.36858971, 4.54830806, 0.29265981],
       [6.46370772, 4.5543959 , 0.28693223],
       [6.47802778, 4.65070459, 0.40397941],
       [6.29717109, 4.55135968, 0.28950038],
       [6.34149378, 4.58114838, 0.42872859],
       [6.1789658 , 4.6589119 , 0.42126921]])

In [106]:
rob_pos,_=tf_man.getTF('base_link')
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))
ix


[ WARN] [1710473773.101862197]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.101925102]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.101952618]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.101978752]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.102010881]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.102037519]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator


4

[ WARN] [1710473773.290014351]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290070224]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290111760]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290147588]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290181250]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290212982]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.74

[ WARN] [1710473776.715985886]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716047074]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716083172]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716109801]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716134253]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716159524]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473778.05

obj_name    spatula
category    kitchen
Name: 6, dtype: object

[ WARN] [1710473806.863826319]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863869473]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863894426]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863932673]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863964885]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863994091]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473807.09

[ WARN] [1710473810.481276302]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481338679]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481390508]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481440216]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481483740]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481528618]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473811.05

[ WARN] [1710473814.770515112]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770570099]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770602751]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770648316]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770683081]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770721679]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473815.20

[ WARN] [1710473818.818562271]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818614164]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818654467]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818690759]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818728534]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818763216]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473819.23

...

[ WARN] [1710473890.908382378]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908437390]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908473499]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908521639]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908559844]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908597533]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473892.06

[ WARN] [1710473897.962512149]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962573494]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962618521]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962659185]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962703576]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962755295]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473898.80

[ WARN] [1710473902.428122221]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428172153]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428197732]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428222960]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428248417]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428272107]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.54

[ WARN] [1710473905.507204782]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507270158]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507307681]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507346708]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507377484]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507410480]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.77

[ WARN] [1710473908.294075612]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294122461]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294151826]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294178525]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294206591]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294233846]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.55

In [111]:
rob_pos,_=tf_man.getTF('base_link')
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))
objs[objs['pickup']==True][['obj_name','category']].iloc[ix]


obj_name    spatula
category    kitchen
Name: 6, dtype: object

In [112]:
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))

In [13]:
pose=np.asarray((8.212561, 6.028789, 0.12))
pose

array([8.212561, 6.028789, 0.12    ])

In [14]:
tf_man.pub_static_tf(pos=pose,point_name='PLACING')

[ WARN] [1710721819.982457468]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982510844]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982544395]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982569973]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982593795]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982616974]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721833.81

[ WARN] [1710721890.117906049]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.117971106]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.118015771]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.118073132]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.118106900]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.118132299]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721895.90

In [2]:
#pose=np.asarray((8.212561, 6.028789, 0.12))
pose=np.asarray((8.062455, 6.0622816, 0.6610926))
tf_man.pub_static_tf(pos=pose,point_name='PLACING')

[ WARN] [1710781382.841518748]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841577977]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841610689]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841640915]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841670497]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841699811]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator


shutdown request: [/smach] Reason: new node registered with same name


In [5]:
arm['arm_lift_joint']=0.0

TypeError: 'MoveGroupCommander' object does not support item assignment

[ WARN] [1710895250.150102700, 1194.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1194.932000 according to authority /pose_integrator
[ WARN] [1710895250.150173990, 1194.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1194.932000 according to authority /pose_integrator
[ WARN] [1710895250.150214698, 1194.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1194.932000 according to authority /pose_integrator
[ WARN] [1710895250.150259279, 1194.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1194.932000 according to authority /pose_integrator
[ WARN] [1710895250.150322160, 1194.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1194.932000 according to authority /pose_integrator
[ WARN] [1710895250.150386765, 1194.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

[ WARN] [1710895255.058572068, 1196.757000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1196.782000 according to authority /pose_integrator
[ WARN] [1710895255.058630610, 1196.757000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1196.782000 according to authority /pose_integrator
[ WARN] [1710895255.058676016, 1196.757000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1196.782000 according to authority /pose_integrator
[ WARN] [1710895255.058714873, 1196.757000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1196.782000 according to authority /pose_integrator
[ WARN] [1710895255.058754969, 1196.757000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1196.782000 according to authority /pose_integrator
[ WARN] [1710895255.058791232, 1196.757000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

[ WARN] [1710895259.723302812, 1198.602000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1198.602000 according to authority /pose_integrator
[ WARN] [1710895259.723386472, 1198.602000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1198.602000 according to authority /pose_integrator
[ WARN] [1710895259.723422004, 1198.602000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1198.602000 according to authority unknown_publisher
[ WARN] [1710895259.724227763, 1198.602000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1198.602000 according to authority /pose_integrator
[ WARN] [1710895259.724304016, 1198.602000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1198.602000 according to authority /pose_integrator
[ WARN] [1710895259.724339795, 1198.602000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

In [6]:
omni_base.move_base(known_location='shelf')

header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
goal_id: 
  stamp: 
    secs: 0
    nsecs:         0
  id: ''
goal: 
  x: 0.0
  y: 0.0
  yaw: 0.0
  timeout: 60
  known_location: "shelf"


[ERROR] [1710895284.213040, 1208.473000]: Got a transition callback on a goal handle that we're not tracking
[ WARN] [1710895284.452660374, 1208.582000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1208.582000 according to authority /pose_integrator
[ WARN] [1710895284.452752920, 1208.582000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1208.582000 according to authority /pose_integrator
[ WARN] [1710895284.452825099, 1208.582000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1208.582000 according to authority /pose_integrator
[ WARN] [1710895284.452890188, 1208.582000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1208.582000 according to authority /pose_integrator
[ WARN] [1710895284.452952561, 1208.582000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1208.582000 according to authority /pose_integra

[ WARN] [1710895289.496286293, 1210.423000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1210.470000 according to authority /pose_integrator
[ WARN] [1710895289.496378614, 1210.423000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1210.470000 according to authority /pose_integrator
[ WARN] [1710895289.496461849, 1210.423000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1210.470000 according to authority /pose_integrator
[ WARN] [1710895289.496528264, 1210.423000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1210.470000 according to authority /pose_integrator
[ WARN] [1710895289.496580246, 1210.423000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1210.470000 according to authority /pose_integrator
[ WARN] [1710895289.496627798, 1210.423000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

[ WARN] [1710895298.794746679, 1213.513000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1213.549000 according to authority /pose_integrator
[ WARN] [1710895298.794829125, 1213.513000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1213.549000 according to authority /pose_integrator
[ WARN] [1710895298.794877990, 1213.513000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1213.549000 according to authority /pose_integrator
[ WARN] [1710895298.794922581, 1213.513000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1213.549000 according to authority /pose_integrator
[ WARN] [1710895298.794971643, 1213.513000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1213.549000 according to authority /pose_integrator
[ WARN] [1710895298.795017518, 1213.513000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

[ WARN] [1710895305.824477630, 1215.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1215.932000 according to authority unknown_publisher
[ WARN] [1710895305.827381970, 1215.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1215.932000 according to authority /pose_integrator
[ WARN] [1710895305.827456440, 1215.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1215.932000 according to authority /pose_integrator
[ WARN] [1710895305.827515767, 1215.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1215.932000 according to authority /pose_integrator
[ WARN] [1710895305.827571212, 1215.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1215.932000 according to authority /pose_integrator
[ WARN] [1710895305.827637526, 1215.932000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

...

[ WARN] [1710895343.414348927, 1229.382000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1229.407000 according to authority unknown_publisher
[ WARN] [1710895343.415211094, 1229.382000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1229.407000 according to authority /pose_integrator
[ WARN] [1710895343.415275586, 1229.382000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1229.407000 according to authority /pose_integrator
[ WARN] [1710895343.415372094, 1229.382000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1229.407000 according to authority /pose_integrator
[ WARN] [1710895343.415432160, 1229.382000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1229.407000 according to authority /pose_integrator
[ WARN] [1710895343.415503427, 1229.382000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1710895346.827118630, 1230.851000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1230.904000 according to authority unknown_publisher
[ WARN] [1710895346.827674770, 1230.851000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1230.904000 according to authority /pose_integrator
[ WARN] [1710895346.827747975, 1230.851000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1230.904000 according to authority /pose_integrator
[ WARN] [1710895346.827806884, 1230.851000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1230.904000 according to authority /pose_integrator
[ WARN] [1710895346.827854849, 1230.851000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1230.904000 according to authority /pose_integrator
[ WARN] [1710895346.827919273, 1230.851000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1710895351.643034032, 1232.809000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1232.829000 according to authority unknown_publisher
[ WARN] [1710895351.643996939, 1232.809000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1232.829000 according to authority /pose_integrator
[ WARN] [1710895351.644069609, 1232.809000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1232.829000 according to authority /pose_integrator
[ WARN] [1710895351.644121399, 1232.809000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1232.829000 according to authority /pose_integrator
[ WARN] [1710895351.644169228, 1232.809000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1232.829000 according to authority /pose_integrator
[ WARN] [1710895351.644215882, 1232.809000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1710895358.294192058, 1235.508000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1235.533000 according to authority unknown_publisher
[ WARN] [1710895358.296039972, 1235.508000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1235.533000 according to authority /pose_integrator
[ WARN] [1710895358.296125701, 1235.508000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1235.533000 according to authority /pose_integrator
[ WARN] [1710895358.296186210, 1235.508000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1235.533000 according to authority /pose_integrator
[ WARN] [1710895358.296244572, 1235.508000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1235.533000 according to authority /pose_integrator
[ WARN] [1710895358.296313999, 1235.508000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1710895362.000414778, 1237.097000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1237.123000 according to authority /pose_integrator
[ WARN] [1710895362.000506105, 1237.097000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1237.123000 according to authority /pose_integrator
[ WARN] [1710895362.000572409, 1237.097000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1237.123000 according to authority /pose_integrator
[ WARN] [1710895362.000634427, 1237.097000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1237.123000 according to authority /pose_integrator
[ WARN] [1710895362.000693306, 1237.097000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1237.123000 according to authority /pose_integrator
[ WARN] [1710895362.000738195, 1237.097000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

In [19]:
omni_base.tiny_move(velT=-0.7,std_time=1)

[ WARN] [1710895997.467298095, 1493.339000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1493.365000 according to authority unknown_publisher
[ WARN] [1710895997.467392500, 1493.339000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1493.365000 according to authority /pose_integrator
[ WARN] [1710895997.467469520, 1493.339000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1493.365000 according to authority /pose_integrator
[ WARN] [1710895997.467539157, 1493.339000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1493.365000 according to authority /pose_integrator
[ WARN] [1710895997.467601930, 1493.339000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1493.365000 according to authority /pose_integrator
[ WARN] [1710895997.467671844, 1493.339000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1710896017.065771287, 1500.444000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1500.472000 according to authority /pose_integrator
[ WARN] [1710896017.065859391, 1500.444000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1500.472000 according to authority /pose_integrator
[ WARN] [1710896017.065925999, 1500.444000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1500.472000 according to authority /pose_integrator
[ WARN] [1710896017.065990538, 1500.444000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1500.472000 according to authority /pose_integrator
[ WARN] [1710896017.066133021, 1500.444000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1500.472000 according to authority unknown_publisher
[ WARN] [1710896017.067342670, 1500.444000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1710896027.503254879, 1504.481000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1504.510000 according to authority unknown_publisher
[ WARN] [1710896027.509155307, 1504.510000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1504.510000 according to authority /pose_integrator
[ WARN] [1710896027.509247207, 1504.510000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1504.510000 according to authority /pose_integrator
[ WARN] [1710896027.509292650, 1504.510000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1504.510000 according to authority /pose_integrator
[ WARN] [1710896027.509346371, 1504.510000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1504.510000 according to authority /pose_integrator
[ WARN] [1710896027.509389371, 1504.510000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

In [21]:
av=arm.get_current_joint_values()
high_shelf_place=av
high_shelf_place


[0.15050079265248406,
 -0.5861087653247212,
 0.0850017715788356,
 -0.9342371117035135,
 0.022035143687695324,
 0.0]

[ WARN] [1710896051.520902312, 1513.606000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1513.647000 according to authority /pose_integrator
[ WARN] [1710896051.520967817, 1513.606000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1513.647000 according to authority /pose_integrator
[ WARN] [1710896051.521015930, 1513.606000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1513.647000 according to authority /pose_integrator
[ WARN] [1710896051.521056876, 1513.606000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1513.647000 according to authority /pose_integrator
[ WARN] [1710896051.521100279, 1513.606000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1513.647000 according to authority /pose_integrator
[ WARN] [1710896051.521147685, 1513.606000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

In [7]:
place_high_shelf=np.asarray([0.15050079265248406,
 -0.5861087653247212,
 0.0850017715788356,
 -0.9342371117035135,
 0.022035143687695324,
 0.0])

[ WARN] [1710901709.981730082, 355.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 355.318000 according to authority unknown_publisher
[ WARN] [1710901709.991425656, 355.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 355.318000 according to authority /pose_integrator
[ WARN] [1710901709.991519786, 355.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 355.318000 according to authority /pose_integrator
[ WARN] [1710901709.991579486, 355.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 355.318000 according to authority /pose_integrator
[ WARN] [1710901709.991628807, 355.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 355.318000 according to authority /pose_integrator
[ WARN] [1710901709.991677597, 355.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710901714.544258872, 357.123000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 357.162000 according to authority unknown_publisher
[ WARN] [1710901714.545178119, 357.123000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 357.162000 according to authority /pose_integrator
[ WARN] [1710901714.545256780, 357.123000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 357.162000 according to authority /pose_integrator
[ WARN] [1710901714.545310034, 357.123000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 357.162000 according to authority /pose_integrator
[ WARN] [1710901714.545371491, 357.123000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 357.162000 according to authority /pose_integrator
[ WARN] [1710901714.545441627, 357.123000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710901717.951333552, 358.686000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 358.714000 according to authority /pose_integrator
[ WARN] [1710901717.951407682, 358.686000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 358.714000 according to authority /pose_integrator
[ WARN] [1710901717.951470732, 358.686000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 358.714000 according to authority /pose_integrator
[ WARN] [1710901717.951519488, 358.686000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 358.714000 according to authority /pose_integrator
[ WARN] [1710901717.951570954, 358.686000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 358.714000 according to authority /pose_integrator
[ WARN] [1710901717.951620917, 358.686000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

[ WARN] [1710901720.138712844, 359.625000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 359.675000 according to authority /pose_integrator
[ WARN] [1710901720.138790246, 359.625000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 359.675000 according to authority /pose_integrator
[ WARN] [1710901720.138846978, 359.625000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 359.675000 according to authority /pose_integrator
[ WARN] [1710901720.138903942, 359.625000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 359.675000 according to authority /pose_integrator
[ WARN] [1710901720.138958103, 359.625000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 359.675000 according to authority /pose_integrator
[ WARN] [1710901720.139005135, 359.625000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

[ WARN] [1710901722.693323931, 360.788000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 360.827000 according to authority unknown_publisher
[ WARN] [1710901722.693438672, 360.788000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 360.827000 according to authority /pose_integrator
[ WARN] [1710901722.693498794, 360.788000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 360.827000 according to authority /pose_integrator
[ WARN] [1710901722.693551675, 360.788000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 360.827000 according to authority /pose_integrator
[ WARN] [1710901722.693598017, 360.788000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 360.827000 according to authority /pose_integrator
[ WARN] [1710901722.693660006, 360.788000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

In [8]:
arm.go(place_high_shelf)

[ WARN] [1710901730.012209195, 363.993000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 364.051000 according to authority unknown_publisher
[ WARN] [1710901730.013767059, 363.993000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 364.051000 according to authority /pose_integrator
[ WARN] [1710901730.013837130, 363.993000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 364.051000 according to authority /pose_integrator
[ WARN] [1710901730.013878693, 363.993000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 364.051000 according to authority /pose_integrator
[ WARN] [1710901730.013915574, 363.993000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 364.051000 according to authority /pose_integrator
[ WARN] [1710901730.013952148, 363.993000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710901735.764050063, 366.515000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 366.574000 according to authority /pose_integrator
[ WARN] [1710901735.764158727, 366.515000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 366.574000 according to authority /pose_integrator
[ WARN] [1710901735.764229530, 366.515000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 366.574000 according to authority /pose_integrator
[ WARN] [1710901735.764310504, 366.515000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 366.574000 according to authority /pose_integrator
[ WARN] [1710901735.764391068, 366.515000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 366.574000 according to authority /pose_integrator
[ WARN] [1710901735.764469320, 366.515000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

[ WARN] [1710901739.691842611, 368.286000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.319000 according to authority /pose_integrator
[ WARN] [1710901739.691905616, 368.286000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.319000 according to authority /pose_integrator
[ WARN] [1710901739.691942096, 368.286000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.319000 according to authority /pose_integrator
[ WARN] [1710901739.691973929, 368.286000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.319000 according to authority /pose_integrator
[ WARN] [1710901739.692011369, 368.286000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.319000 according to authority /pose_integrator
[ WARN] [1710901739.692043358, 368.286000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

True

[ WARN] [1710901741.109316736, 368.906000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.969000 according to authority /pose_integrator
[ WARN] [1710901741.109394581, 368.906000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.969000 according to authority /pose_integrator
[ WARN] [1710901741.109451241, 368.906000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.969000 according to authority /pose_integrator
[ WARN] [1710901741.109502306, 368.906000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.969000 according to authority /pose_integrator
[ WARN] [1710901741.109585372, 368.906000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 368.969000 according to authority /pose_integrator
[ WARN] [1710901741.109633855, 368.906000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

...

[ WARN] [1710902095.180724609, 519.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 519.305000 according to authority unknown_publisher
[ WARN] [1710902095.180969074, 519.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 519.305000 according to authority /pose_integrator
[ WARN] [1710902095.181049900, 519.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 519.305000 according to authority /pose_integrator
[ WARN] [1710902095.181100461, 519.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 519.305000 according to authority /pose_integrator
[ WARN] [1710902095.181151842, 519.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 519.305000 according to authority /pose_integrator
[ WARN] [1710902095.181449565, 519.274000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710902100.780809368, 521.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 521.262000 according to authority /pose_integrator
[ WARN] [1710902100.780872565, 521.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 521.262000 according to authority /pose_integrator
[ WARN] [1710902100.780916565, 521.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 521.262000 according to authority /pose_integrator
[ WARN] [1710902100.780957840, 521.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 521.262000 according to authority /pose_integrator
[ WARN] [1710902100.781002325, 521.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 521.262000 according to authority /pose_integrator
[ WARN] [1710902100.781040551, 521.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

[ WARN] [1710902105.203233473, 522.812000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 522.849000 according to authority unknown_publisher
[ WARN] [1710902105.204893013, 522.812000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 522.849000 according to authority /pose_integrator
[ WARN] [1710902105.204949116, 522.812000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 522.849000 according to authority /pose_integrator
[ WARN] [1710902105.204979979, 522.812000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 522.849000 according to authority /pose_integrator
[ WARN] [1710902105.205009689, 522.812000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 522.849000 according to authority /pose_integrator
[ WARN] [1710902105.205052581, 522.812000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710902109.366824831, 524.338000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 524.378000 according to authority unknown_publisher
[ WARN] [1710902109.367596000, 524.338000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 524.378000 according to authority /pose_integrator
[ WARN] [1710902109.367676515, 524.338000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 524.378000 according to authority /pose_integrator
[ WARN] [1710902109.367746502, 524.338000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 524.378000 according to authority /pose_integrator
[ WARN] [1710902109.367809728, 524.338000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 524.378000 according to authority /pose_integrator
[ WARN] [1710902109.367850659, 524.338000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710902116.329007036, 526.887000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 526.910000 according to authority /pose_integrator
[ WARN] [1710902116.329093077, 526.887000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 526.910000 according to authority /pose_integrator
[ WARN] [1710902116.329139529, 526.887000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 526.910000 according to authority /pose_integrator
[ WARN] [1710902116.329200257, 526.887000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 526.910000 according to authority /pose_integrator
[ WARN] [1710902116.329244280, 526.887000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 526.910000 according to authority /pose_integrator
[ WARN] [1710902116.329296173, 526.887000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

av=arm.get_current_joint_values()


In [4]:
av=arm.get_current_joint_values()
high_shelf_place=av

In [13]:
arm.go(high_shelf_place)

[ WARN] [1710902460.618572798, 668.800000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 668.832000 according to authority unknown_publisher
[ WARN] [1710902460.619446721, 668.800000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 668.832000 according to authority /pose_integrator
[ WARN] [1710902460.619503119, 668.800000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 668.832000 according to authority /pose_integrator
[ WARN] [1710902460.619550712, 668.800000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 668.832000 according to authority /pose_integrator
[ WARN] [1710902460.619595039, 668.800000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 668.832000 according to authority /pose_integrator
[ WARN] [1710902460.619639942, 668.800000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710902464.016404119, 670.298000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 670.334000 according to authority /pose_integrator
[ WARN] [1710902464.018116319, 670.298000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 670.334000 according to authority /pose_integrator
[ WARN] [1710902464.018656615, 670.298000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 670.334000 according to authority /pose_integrator
[ WARN] [1710902464.018707045, 670.298000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 670.334000 according to authority /pose_integrator
[ WARN] [1710902464.018746944, 670.298000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 670.334000 according to authority /pose_integrator
[ WARN] [1710902464.019105051, 670.298000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

[ WARN] [1710902467.235997700, 671.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 671.783000 according to authority unknown_publisher
[ WARN] [1710902467.237550246, 671.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 671.783000 according to authority /pose_integrator
[ WARN] [1710902467.237624002, 671.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 671.783000 according to authority /pose_integrator
[ WARN] [1710902467.237676648, 671.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 671.783000 according to authority /pose_integrator
[ WARN] [1710902467.237734539, 671.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 671.783000 according to authority /pose_integrator
[ WARN] [1710902467.237787360, 671.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710902469.869519220, 672.933000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 672.974000 according to authority /pose_integrator
[ WARN] [1710902469.869599465, 672.933000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 672.974000 according to authority /pose_integrator
[ WARN] [1710902469.869660657, 672.933000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 672.974000 according to authority /pose_integrator
[ WARN] [1710902469.869712133, 672.933000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 672.974000 according to authority /pose_integrator
[ WARN] [1710902469.869770492, 672.933000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 672.974000 according to authority /pose_integrator
[ WARN] [1710902469.869815809, 672.933000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

[ WARN] [1710902473.155407231, 674.397000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 674.428000 according to authority unknown_publisher
[ WARN] [1710902473.156588195, 674.397000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 674.428000 according to authority /pose_integrator
[ WARN] [1710902473.156655537, 674.397000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 674.428000 according to authority /pose_integrator
[ WARN] [1710902473.156714190, 674.397000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 674.428000 according to authority /pose_integrator
[ WARN] [1710902473.156760526, 674.397000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 674.428000 according to authority /pose_integrator
[ WARN] [1710902473.156803501, 674.397000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

...

[ WARN] [1710902506.918365643, 688.959000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 688.990000 according to authority /pose_integrator
[ WARN] [1710902506.918432526, 688.959000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 688.990000 according to authority /pose_integrator
[ WARN] [1710902506.918475236, 688.959000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 688.990000 according to authority /pose_integrator
[ WARN] [1710902506.918513334, 688.959000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 688.990000 according to authority /pose_integrator
[ WARN] [1710902506.918549966, 688.959000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 688.990000 according to authority /pose_integrator
[ WARN] [1710902506.918587437, 688.959000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

[ WARN] [1710902510.404497325, 690.550000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 690.572000 according to authority unknown_publisher
[ WARN] [1710902510.404592215, 690.550000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 690.572000 according to authority /pose_integrator
[ WARN] [1710902510.404681237, 690.550000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 690.572000 according to authority /pose_integrator
[ WARN] [1710902510.404744313, 690.550000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 690.572000 according to authority /pose_integrator
[ WARN] [1710902510.404798102, 690.550000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 690.572000 according to authority /pose_integrator
[ WARN] [1710902510.404848936, 690.550000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710902515.088427276, 692.457000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 692.486000 according to authority unknown_publisher
[ WARN] [1710902515.098812318, 692.479000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 692.486000 according to authority /pose_integrator
[ WARN] [1710902515.098923132, 692.486000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 692.486000 according to authority /pose_integrator
[ WARN] [1710902515.098992938, 692.486000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 692.486000 according to authority /pose_integrator
[ WARN] [1710902515.099054778, 692.486000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 692.486000 according to authority /pose_integrator
[ WARN] [1710902515.099107680, 692.486000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710902520.787333674, 694.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 694.991000 according to authority unknown_publisher
[ WARN] [1710902520.787970147, 694.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 694.991000 according to authority /pose_integrator
[ WARN] [1710902520.788052872, 694.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 694.991000 according to authority /pose_integrator
[ WARN] [1710902520.788124780, 694.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 694.991000 according to authority /pose_integrator
[ WARN] [1710902520.788162703, 694.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 694.991000 according to authority /pose_integrator
[ WARN] [1710902520.788199316, 694.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710902522.372689929, 695.668000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 695.698000 according to authority /pose_integrator
[ WARN] [1710902522.372764928, 695.668000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 695.698000 according to authority /pose_integrator
[ WARN] [1710902522.372822345, 695.668000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 695.698000 according to authority /pose_integrator
[ WARN] [1710902522.372878303, 695.668000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 695.698000 according to authority /pose_integrator
[ WARN] [1710902522.372936481, 695.668000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 695.698000 according to authority /pose_integrator
[ WARN] [1710902522.372986970, 695.668000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame od

In [3]:
av=arm.get_current_joint_values()
mid_shelf_place=av
mid_shelf_place

[0.2556342351169577,
 -1.6040815280188854,
 -0.008078345343693893,
 -0.05795663028496589,
 0.0019971082238159354,
 0.0]

In [5]:
av=arm.get_current_joint_values()
low_shelf_place=av
low_shelf_place

[0.04577467546491997,
 -2.262541528018885,
 0.00010865465630605087,
 0.701618369715034,
 -0.0003068917761841483,
 0.0]

In [18]:
high_shelf_place,low_shelf_place,mid_shelf_place

([0.4337121304669095,
  -1.3139266720690426,
  0.08621383272112926,
  -0.4260850221072081,
  0.02285051353933021,
  0.0],
 [-5.465250513158978e-07,
  -1.8875162206087301,
  0.08646571240675005,
  0.3040738848075222,
  0.022992974203400962,
  0.0],
 [0.14251866363943907,
  -1.3179204560134075,
  0.08645408715269554,
  -0.42863767361691707,
  0.02300980666974084,
  0.0])

[ WARN] [1710902658.568209776, 753.941000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 753.978000 according to authority unknown_publisher
[ WARN] [1710902658.575309001, 753.979000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 753.978000 according to authority /pose_integrator
[ WARN] [1710902658.575388963, 753.979000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 753.978000 according to authority /pose_integrator
[ WARN] [1710902658.575452140, 753.979000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 753.978000 according to authority /pose_integrator
[ WARN] [1710902658.575497752, 753.979000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 753.978000 according to authority /pose_integrator
[ WARN] [1710902658.575545547, 753.979000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710902661.766320719, 755.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 755.289000 according to authority /pose_integrator
[ WARN] [1710902661.766388396, 755.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 755.289000 according to authority /pose_integrator
[ WARN] [1710902661.766419942, 755.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 755.289000 according to authority unknown_publisher
[ WARN] [1710902661.766941695, 755.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 755.289000 according to authority /pose_integrator
[ WARN] [1710902661.767026577, 755.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 755.289000 according to authority /pose_integrator
[ WARN] [1710902661.767097942, 755.241000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

In [19]:
high_shelf_place=np.asarray([0.4337121304669095,
 -1.3139266720690426,
 0.08621383272112926,
 -0.4260850221072081,
 0.02285051353933021,
 0.0])

[0.4337121304669095,
 -1.3139266720690426,
 0.08621383272112926,
 -0.4260850221072081,
 0.02285051353933021,
 0.0]

In [21]:
low_shelf_place=np.asarray([-5.465250513158978e-07,
 -1.8875162206087301,
 0.08646571240675005,
 0.3040738848075222,
 0.022992974203400962,
 0.0]
)

In [6]:
high_shelf_place=np.asarray([0.15050079265248406,
 -0.5861087653247212,
 0.0850017715788356,
 -0.9342371117035135,
 0.022035143687695324,
 0.0])
mid_shelf_place=np.asarray([0.2556342351169577,
 -1.6040815280188854,
 -0.008078345343693893,
 -0.05795663028496589,
 0.0019971082238159354,
 0.0])

low_shelf_place=np.asarray([0.04577467546491997,
 -2.262541528018885,
 0.00010865465630605087,
 0.701618369715034,
 -0.0003068917761841483,
 0.0]
)